

Nội dung bài thực hành

Người học tiếp cận và giải quyết bài toán phân tích cảm xúc sử dụng mô hình LSTM có dùng các mô hình wordvector khác nhau. Sau khi thực hành, người học có khả năng:


* Huấn luyện và đánh giá mô hình phân loại cảm xúc sử dụng LSTM
* Sử dụng và so sánh nhanh các mô hình từ nhúng thông dụng:
    * Khởi tạo ngẫu nhiên hoàn toàn
    * Sử dụng vector huấn luyện sẵn của mô hình word2vec
    * Sử dụng vector huấn luyện sẵn của mô hình Glove
    * Sử dụng vector huấn luyện sẵn của mô hình FastText

Thao tác với dữ liệu

    Bài thực hành sử dụng bộ dữ liệu IMDB review
    Dữ liệu gồm 2 phần: tập huấn luyện và tập kiểm thử (train data và test data). Mỗi phần gồm có 25000 câu, đuợc phân vào 2 lớp cảm xúc tích cực (positive) và tiêu cực (negative)
    Yêu cầu: xây dựng mô hình phân tích cảm xúc các đánh giá của nguời dùng dựa trên LSTM



In [1]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2019-04-20 14:17:08--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: 'aclImdb_v1.tar.gz'

aclImdb_v1.tar.gz   100%[===================>]  80.23M  2.98MB/s    in 42s     

2019-04-20 14:17:51 (1.93 MB/s) - 'aclImdb_v1.tar.gz' saved [84125825/84125825]



In [2]:
# unzip downloaded file
import tarfile
tf = tarfile.open("aclImdb_v1.tar.gz")
tf.extractall()

In [2]:
# Merge file
# test
import pandas as pd
import os
labels={"pos":1, "neg":0}
test_df = pd.DataFrame()
train_df = pd.DataFrame()

for l in ("pos", "neg"):
    path = "aclImdb/train/%s" %(l)
    for file in os.listdir(path):
        with open(os.path.join(path, file)) as infile:
            txt = infile.read()
        train_df = train_df.append([[txt, labels[l]]], ignore_index=True)
train_df.columns=["review", "sentiment"]

for l in ("pos", "neg"):
    path = "aclImdb/test/%s" %(l)
    for file in os.listdir(path):
        with open(os.path.join(path, file)) as infile:
            txt = infile.read()
        test_df = test_df.append([[txt, labels[l]]], ignore_index=True)
test_df.columns=["review", "sentiment"]

In [3]:
import re
def preprocessor(text):
    text = re.sub(r"<[^>]>", '', text)
    emoticons = re.findall("(?:|;|=)(?:-)?(?:\)\(|D|P)", text)
    text = re.sub("[\W]+", " ", text.lower()) + \
            " ".join(emoticons).replace('-', '')
    return text.lower()
train_df["review"] = train_df["review"].apply(preprocessor)
test_df["review"] = test_df["review"].apply(preprocessor)

### Thiet Lap tham so cho mo hinh

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.recurrent import LSTM
from keras.layers import CuDNNLSTM # comment dong nay neu may ban khong ho tro CuDNN
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.layers import Input, TimeDistributed, BatchNormalization
from keras.layers.core import Activation, Dense, Dropout
from keras.layers.wrappers import Bidirectional

from keras import backend as K


max_len = 500
num_words = 5000
embedding_dim = 300
dropout = 0.5
hidden_dim = 750
l2_reg = 1e-4
batch_size = 160
epochs = 5
learning_rate = 1e-3
rnnact = 'tanh'
lstm = CuDNNLSTM
opt = 'adadelta'

# embedding
word2vec_file = 'GoogleNews-vectors-negative300.bin'
glove_file = 'glove.6B.300d.txt'
fasttext_file = 'wiki-news-300d-1M.vec'


Using TensorFlow backend.


In [5]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_df["review"])

In [6]:
X_train = tokenizer.texts_to_sequences(train_df["review"])
X_test = tokenizer.texts_to_sequences(test_df["review"])
X_train = pad_sequences(X_train, maxlen=500)
X_test = pad_sequences(X_test, maxlen=500)

In [7]:
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
import numpy as np

def get_embedded(wordvector):
    word_exits = 0
    vocab = tokenizer.index_word
    embedded_matrix = np.zeros((5000,300))
    print("Doc wordvector tu %s ..." %wordvector)
    if wordvector == "word2vec":
        model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    elif wordvector == "glove":
        model = {}
        with open('glove.6B.300d.txt', 'r') as f:
            for line in f:
                l = line.split()
                word = l[0]
                model[word] = np.array(l[1:]).astype(float)
    elif wordvector == "fasttext":
        model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')

    for word in vocab:
        if word > 4999:
            break
        try:
            embedded_matrix[word, :] = model[vocab[word]]
            word_exits += 1
        except KeyError:
            if word == 0:
                embedded_matrix[word, :] = np.zeros(300)
            else:
                # 0.25 is embedding SD
                embedded_matrix[word, :] = np.random.uniform(-0.25, 0.25, 300)
    print("Found %s word in embedding file" %word_exits)
    return embedded_matrix

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [8]:
def get_model(embedding="random"):
    input_s = Input(name='input_si', shape=(max_len,), dtype='int32')
    if embedding == "random":
        shared_embedding = Embedding(name='emb', input_dim=num_words, input_length=max_len,
                                output_dim=embedding_dim, mask_zero=False, trainable=True)
    else:
        embedded_matrix = get_embedded(embedding)
        shared_embedding = Embedding(name='emb', input_dim=num_words, input_length=max_len,
                                     weights=[embedded_matrix],output_dim=embedding_dim, mask_zero=False, trainable=True)
    s_embedding = shared_embedding(input_s)
    s_embedding = Dropout(dropout)(s_embedding)
    shared_lstm = Bidirectional(
            lstm(hidden_dim, input_shape=(None, 500, 300),return_sequences=False, name='rnn'),
            merge_mode='concat')
    s_lstm = shared_lstm(s_embedding)
    s_lstm = Dropout(dropout)(s_lstm)
    yhat = Dense(1, activation="sigmoid")(s_lstm)
    model = Model(inputs=input_s, outputs=yhat)
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer="adam")
    return model

### Mô hình sử dụng wordvector sinh ngẫu nhiên

In [9]:
random_model = get_model()

In [10]:
random_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_si (InputLayer)        (None, 500)               0         
_________________________________________________________________
emb (Embedding)              (None, 500, 300)          1500000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1500)              6312000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1501      
Total params: 7,813,501
Trainable params: 7,813,501
Non-trainable params: 0
_________________________________________________________________


In [11]:
random_model.fit(X_train, train_df["sentiment"], validation_split=0.2, batch_size=batch_size, epochs=epochs)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 91s 5ms/step - loss: 0.5941 - acc: 0.6943 - val_loss: 0.5625 - val_acc: 0.7008
Epoch 2/5
20000/20000 [==============================] - 85s 4ms/step - loss: 0.4551 - acc: 0.8047 - val_loss: 0.5931 - val_acc: 0.7778
Epoch 3/5
20000/20000 [==============================] - 90s 5ms/step - loss: 0.3576 - acc: 0.8526 - val_loss: 0.4144 - val_acc: 0.8182
Epoch 4/5
20000/20000 [==============================] - 91s 5ms/step - loss: 0.3209 - acc: 0.8757 - val_loss: 0.5663 - val_acc: 0.7940
Epoch 5/5
20000/20000 [==============================] - 89s 4ms/step - loss: 0.2673 - acc: 0.8987 - val_loss: 0.8017 - val_acc: 0.7140


In [12]:
random_model.evaluate(X_test, test_df["sentiment"])

25000/25000 [==============================] - 49s 2ms/step


[0.4713997391295433, 0.8296]

### Mo6 hình sử dụng wordvector huấn luyện sẵn từ  Word2vec

In [13]:
K.clear_session()
word2vec_model = get_model('word2vec')

Doc wordvector tu word2vec ...
Found 4842 word in embedding file


In [14]:
word2vec_model.fit(X_train, train_df["sentiment"], validation_split=0.2, batch_size=batch_size, epochs=epochs)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 85s 4ms/step - loss: 0.6800 - acc: 0.6179 - val_loss: 0.8094 - val_acc: 0.0012
Epoch 2/5
20000/20000 [==============================] - 85s 4ms/step - loss: 0.6136 - acc: 0.6673 - val_loss: 1.0790 - val_acc: 0.2312
Epoch 3/5
20000/20000 [==============================] - 90s 4ms/step - loss: 0.3466 - acc: 0.8509 - val_loss: 0.5450 - val_acc: 0.7798
Epoch 4/5
20000/20000 [==============================] - 90s 4ms/step - loss: 0.2379 - acc: 0.9073 - val_loss: 0.3816 - val_acc: 0.8446
Epoch 5/5
20000/20000 [==============================] - 90s 4ms/step - loss: 0.3505 - acc: 0.8268 - val_loss: 0.4071 - val_acc: 0.8340


In [15]:
word2vec_model.evaluate(X_test, test_df["sentiment"])

25000/25000 [==============================] - 49s 2ms/step


[0.3082465085411072, 0.87392]

### Mô hình sử dụng wordvector huấn luyện sẵn từ  Glove

In [16]:
K.clear_session()
glove_model = get_model('glove')

Doc wordvector tu glove ...
Found 4998 word in embedding file


In [17]:
glove_model.fit(X_train, train_df["sentiment"], validation_split=0.2, batch_size=batch_size, epochs=epochs)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 85s 4ms/step - loss: 0.6607 - acc: 0.6297 - val_loss: 0.4930 - val_acc: 0.8996
Epoch 2/5
20000/20000 [==============================] - 85s 4ms/step - loss: 0.5202 - acc: 0.7456 - val_loss: 0.4669 - val_acc: 0.8094
Epoch 3/5
20000/20000 [==============================] - 84s 4ms/step - loss: 0.3094 - acc: 0.8729 - val_loss: 0.4278 - val_acc: 0.7848
Epoch 4/5
20000/20000 [==============================] - 87s 4ms/step - loss: 0.2428 - acc: 0.9040 - val_loss: 0.3503 - val_acc: 0.8558
Epoch 5/5
20000/20000 [==============================] - 87s 4ms/step - loss: 0.2117 - acc: 0.9181 - val_loss: 0.2843 - val_acc: 0.8876


In [18]:
glove_model.evaluate(X_test, test_df["sentiment"])

25000/25000 [==============================] - 49s 2ms/step


[0.2501241451025009, 0.89728]

### Mô hình sử dụng wordvector huấn luyện sẵn từ  Fasttext

In [23]:
K.clear_session()
fasttext_model = get_model("fasttext")

Doc wordvector tu fasttext ...
Found 4958 word in embedding file


In [24]:
fasttext_model.fit(X_train, train_df["sentiment"], validation_split=0.2, batch_size=batch_size, epochs=epochs)

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 84s 4ms/step - loss: 0.6434 - acc: 0.6396 - val_loss: 0.6449 - val_acc: 0.6950
Epoch 2/5
20000/20000 [==============================] - 84s 4ms/step - loss: 0.4279 - acc: 0.8130 - val_loss: 0.7855 - val_acc: 0.6330
Epoch 3/5
20000/20000 [==============================] - 85s 4ms/step - loss: 0.3330 - acc: 0.8662 - val_loss: 0.8845 - val_acc: 0.5524
Epoch 4/5
20000/20000 [==============================] - 89s 4ms/step - loss: 0.3054 - acc: 0.8801 - val_loss: 0.6021 - val_acc: 0.7528
Epoch 5/5
20000/20000 [==============================] - 87s 4ms/step - loss: 0.2404 - acc: 0.9071 - val_loss: 0.5164 - val_acc: 0.8056


In [25]:
fasttext_model.evaluate(X_test, test_df["sentiment"])

25000/25000 [==============================] - 49s 2ms/step


[0.3401986590695381, 0.86984]